In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install keras-tuner --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.2 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout
from keras_tuner import HyperModel
from keras_tuner import RandomSearch

In [4]:
movies        = pd.read_csv('/content/drive/MyDrive/KU Module/Project Dissertation/Project_Model/movies.csv')
links         = pd.read_csv('/content/drive/MyDrive/KU Module/Project Dissertation/Project_Model/links.csv')
ratings       = pd.read_csv('/content/drive/MyDrive/KU Module/Project Dissertation/Project_Model/ratings.csv')
tags          = pd.read_csv('/content/drive/MyDrive/KU Module/Project Dissertation/Project_Model/tags.csv')

In [5]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [6]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [7]:
movies.isnull().sum()

,0
movieId,0
title,0
genres,0


In [8]:
print(movies['genres'].nunique())
print(movies['genres'].unique())

951
['Adventure|Animation|Children|Comedy|Fantasy'
 'Adventure|Children|Fantasy' 'Comedy|Romance' 'Comedy|Drama|Romance'
 'Comedy' 'Action|Crime|Thriller' 'Adventure|Children' 'Action'
 'Action|Adventure|Thriller' 'Comedy|Horror'
 'Adventure|Animation|Children' 'Drama' 'Action|Adventure|Romance'
 'Crime|Drama' 'Drama|Romance' 'Action|Comedy|Crime|Drama|Thriller'
 'Comedy|Crime|Thriller' 'Crime|Drama|Horror|Mystery|Thriller'
 'Drama|Sci-Fi' 'Children|Drama' 'Adventure|Drama|Fantasy|Mystery|Sci-Fi'
 'Mystery|Sci-Fi|Thriller' 'Children|Comedy' 'Drama|War'
 'Action|Crime|Drama' 'Action|Adventure|Fantasy' 'Comedy|Drama|Thriller'
 'Mystery|Thriller' 'Animation|Children|Drama|Musical|Romance'
 'Crime|Mystery|Thriller' 'Adventure|Drama' 'Drama|Thriller'
 'Comedy|Crime' 'Action|Sci-Fi|Thriller' 'Action|Comedy|Horror|Thriller'
 'Comedy|Drama' 'Documentary' 'Action|Crime|Drama|Thriller'
 'Crime|Drama|Romance' 'Action|Adventure|Drama' 'Action|Thriller'
 'Drama|Horror|Thriller' 'Comedy|Horror|Roman

In [9]:
links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [10]:
links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9742 non-null   int64  
 1   imdbId   9742 non-null   int64  
 2   tmdbId   9734 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 228.5 KB


In [11]:
links.isnull().sum() # There is some missing values in links dataframe

,0
movieId,0
imdbId,0
tmdbId,8


In [12]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [13]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [14]:
ratings.isnull().sum()

,0
userId,0
movieId,0
rating,0
timestamp,0


In [15]:
ratings['userId'].nunique()

610

In [16]:
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [17]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userId     3683 non-null   int64 
 1   movieId    3683 non-null   int64 
 2   tag        3683 non-null   object
 3   timestamp  3683 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


In [18]:
tags.isnull().sum()

,0
userId,0
movieId,0
tag,0
timestamp,0


In [19]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [20]:
ratings['rating'].unique()

array([4. , 5. , 3. , 2. , 1. , 4.5, 3.5, 2.5, 0.5, 1.5])

# Initial Model Setup and Baseline Performance

## Wide and Deep Model

In [21]:
# Encode userId and movieId as integers
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()

ratings['userId'] = user_encoder.fit_transform(ratings['userId'])
ratings['movieId'] = movie_encoder.fit_transform(ratings['movieId'])

# Ensure num_users and num_movies are based on the entire dataset
num_users = ratings['userId'].max() + 1
num_movies = ratings['movieId'].max() + 1

# Check for any out-of-bounds userId or movieId
max_user_in_train = ratings['userId'].max()
max_movie_in_train = ratings['movieId'].max()

print(f"Max userId in dataset: {max_user_in_train}")
print(f"Max movieId in dataset: {max_movie_in_train}")

# Separate features and target
X_wide = ratings[['userId', 'movieId']]
X_deep = ratings[['userId', 'movieId']]
y = ratings['rating'].values

# Split the data into training and testing sets
X_wide_train, X_wide_test, X_deep_train, X_deep_test, y_train, y_test = train_test_split(X_wide, X_deep, y, test_size=0.2, random_state=42)

# Define input layers
user_input = Input(shape=(1,), name='user_input')
movie_input = Input(shape=(1,), name='movie_input')

# Wide part: Direct feature interactions
wide_input = Concatenate()([user_input, movie_input])

# Deep part: Embeddings for userId and movieId
embedding_dim = 16

user_embedding = Embedding(input_dim=num_users, output_dim=embedding_dim)(user_input)
movie_embedding = Embedding(input_dim=num_movies, output_dim=embedding_dim)(movie_input)

user_vec = Flatten()(user_embedding)
movie_vec = Flatten()(movie_embedding)

deep_input = Concatenate()([user_vec, movie_vec])
deep_output = Dense(128, activation='relu')(deep_input)
deep_output = Dropout(0.3)(deep_output)
deep_output = Dense(64, activation='relu')(deep_output)
deep_output = Dropout(0.3)(deep_output)

# Combine Wide and Deep parts
combined_input = Concatenate()([wide_input, deep_output])
output = Dense(1, activation='linear')(combined_input)

# Define the model
model = Model(inputs=[user_input, movie_input], outputs=output)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='mean_squared_error',
              metrics=['mean_absolute_error'])

# Model summary
model.summary()

Max userId in dataset: 609
Max movieId in dataset: 9723


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ user_input (InputLayer)   │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ movie_input (InputLayer)  │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 1, 16)          │          9,760 │ user_input[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, 1, 16)          │        155,584 │ movie_input[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 16)             │              0 │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_1 (Flatten)       │ (None, 16)             │              0 │ embedding_1[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 32)             │              0 │ flatten[0][0],         │
│ (Concatenate)             │                        │                │ flatten_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │          4,224 │ concatenate_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 128)            │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 64)             │          8,256 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 2)              │              0 │ user_input[0][0],      │
│                           │                        │                │ movie_input[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 64)             │              0 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_2             │ (None, 66)             │              0 │ concatenate[0][0],     │
│ (Concatenate)             │                        │                │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 1)              │             67 │ concatenate_2[0][0]    │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 177,891 (694.89 KB)

 Trainable params: 177,891 (694.89 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
type(ratings['userId'])

pandas.core.series.Series

In [23]:
num_users

610

In [24]:
num_movies

9724

In [25]:
max_user_in_train

609

In [26]:
max_movie_in_train

9723

In [27]:
# Prepare input dictionaries for wide and deep parts
train_dict = {
    'user_input': X_deep_train['userId'].values,
    'movie_input': X_deep_train['movieId'].values
}

test_dict = {
    'user_input': X_deep_test['userId'].values,
    'movie_input': X_deep_test['movieId'].values
}
# Start timer for training time
start_time = time.time()
# Train the model
history = model.fit(train_dict, y_train, epochs=10, batch_size=64, validation_data=(test_dict, y_test))
# End timer for training time
training_time = time.time() - start_time

Epoch 1/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - loss: 1122.8693 - mean_absolute_error: 11.1680 - val_loss: 0.9805 - val_mean_absolute_error: 0.7707
Epoch 2/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 1.1732 - mean_absolute_error: 0.8473 - val_loss: 0.9291 - val_mean_absolute_error: 0.7622
Epoch 3/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.9925 - mean_absolute_error: 0.7804 - val_loss: 0.9371 - val_mean_absolute_error: 0.7687
Epoch 4/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.9618 - mean_absolute_error: 0.7669 - val_loss: 1.0230 - val_mean_absolute_error: 0.7741
Epoch 5/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.9791 - mean_absolute_error: 0.7757 - val_loss: 1.1223 - val_mean_absolute_error: 0.8653
Epoch 6/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.9870 - mean_absolute_error: 0.7767 - val_loss: 0.9143 - val_mean_absolute_error: 0.7437
Epoch 7/10
1261/1261 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.9996 - mean_absolu

In [28]:
# Evaluate the model on the test set
test_loss, test_mae = model.evaluate(test_dict, y_test)

# Calculate RMSE from MSE
test_rmse = np.sqrt(test_loss)
print(f"Training Time: {training_time:.2f} seconds")
print(f"Test MSE: {test_loss:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")
print(f"Test MAE: {test_mae:.4f}")

631/631 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.7182 - mean_absolute_error: 1.0102
Training Time: 63.08 seconds
Test MSE: 1.6756
Test RMSE: 1.2944
Test MAE: 1.0007


In [29]:
# Define a hypermodel for the Wide & Deep Model
class WideAndDeepHyperModel(HyperModel):
    def build(self, hp):
        # Define input layers
        user_input = Input(shape=(1,), name='user_input')
        movie_input = Input(shape=(1,), name='movie_input')

        # Wide part: Direct feature interactions
        wide_input = Concatenate()([user_input, movie_input])

        # Deep part: Embeddings for userId and movieId
        embedding_dim = hp.Int('embedding_dim', min_value=8, max_value=32, step=8)
        user_embedding = Embedding(input_dim=num_users, output_dim=embedding_dim)(user_input)
        movie_embedding = Embedding(input_dim=num_movies, output_dim=embedding_dim)(movie_input)

        user_vec = Flatten()(user_embedding)
        movie_vec = Flatten()(movie_embedding)

        deep_input = Concatenate()([user_vec, movie_vec])
        deep_output = Dense(hp.Int('dense_units_1', min_value=64, max_value=256, step=64), activation='relu')(deep_input)
        deep_output = Dropout(hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1))(deep_output)
        deep_output = Dense(hp.Int('dense_units_2', min_value=32, max_value=128, step=32), activation='relu')(deep_output)
        deep_output = Dropout(hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1))(deep_output)

        # Combine Wide and Deep parts
        combined_input = Concatenate()([wide_input, deep_output])
        output = Dense(1, activation='linear')(combined_input)

        # Define the model
        model = Model(inputs=[user_input, movie_input], outputs=output)

        # Compile the model
        model.compile(optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
            loss='mean_squared_error',
            metrics=['mean_absolute_error'])

        return model

# Instantiate the hypermodel
hypermodel = WideAndDeepHyperModel()

# Instantiate the tuner
tuner = RandomSearch(
    hypermodel,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    directory='wide_and_deep_tuning',
    project_name='movie_recommendation'
)

# Prepare input dictionaries for wide and deep parts
train_dict = {
    'user_input': X_deep_train['userId'].values,
    'movie_input': X_deep_train['movieId'].values
}

test_dict = {
    'user_input': X_deep_test['userId'].values,
    'movie_input': X_deep_test['movieId'].values
}

# Start timer for hyperparameter tuning
start_time = time.time()

# Run the hyperparameter search
tuner.search(
    train_dict, y_train,
    epochs=10,
    batch_size=64,
    validation_data=(test_dict, y_test)
)

# End timer for hyperparameter tuning
tuning_time = time.time() - start_time

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model
test_loss, test_mae = best_model.evaluate(test_dict, y_test)
test_rmse = np.sqrt(test_loss)

# Print the results
print(f"Hyperparameter Tuning Time: {tuning_time:.2f} seconds")
print(f"Best model - Test MSE: {test_loss:.4f}")
print(f"Best model - Test RMSE: {test_rmse:.4f}")
print(f"Best model - Test MAE: {test_mae:.4f}")

# To get the best hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best hyperparameters: {best_hyperparameters.values}")

Trial 10 Complete [00h 00m 44s]
val_loss: 0.9046394228935242

Best val_loss So Far: 0.8893365263938904
Total elapsed time: 00h 07m 55s


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


631/631 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.9030 - mean_absolute_error: 0.7374
Hyperparameter Tuning Time: 475.28 seconds
Best model - Test MSE: 0.8893
Best model - Test RMSE: 0.9430
Best model - Test MAE: 0.7315
Best hyperparameters: {'embedding_dim': 32, 'dense_units_1': 192, 'dropout_rate': 0.2, 'dense_units_2': 128, 'learning_rate': 0.00020253845754746594}


In [ ]:
import numpy as np
import pandas as pd

# Assuming 'best_model' is your trained Wide & Deep model after hyperparameter tuning

# Select the userId for which you want recommendations
user_id = 1

# Get all movieIds
all_movie_ids = np.array(ratings['movieId'].unique())

# Get the movies that user 1 has already rated
rated_movie_ids = ratings[ratings['userId'] == user_id]['movieId'].values

# Find the movies that user 1 has not rated
unrated_movie_ids = np.setdiff1d(all_movie_ids, rated_movie_ids)

# Prepare the input for prediction
user_input = np.full_like(unrated_movie_ids, user_id)
movie_input = unrated_movie_ids

# Create input dictionary for the best model
input_dict = {
    'user_input': user_input,
    'movie_input': movie_input
}

# Predict ratings for all unrated movies in a single pass
predicted_ratings = best_model.predict(input_dict).flatten()

# Clip the predicted ratings to be within the range [1, 5]
predicted_ratings = np.clip(predicted_ratings, 1, 5)

# Get the top 10 movie indices
top_10_indices = np.argsort(predicted_ratings)[-10:][::-1]

# Get the top 10 movieIds and their predicted ratings
top_10_movie_ids = unrated_movie_ids[top_10_indices]
top_10_ratings = predicted_ratings[top_10_indices]

# Display the top 10 recommendations
recommended_movies = pd.DataFrame({
    'movieId': top_10_movie_ids,
    'predicted_rating': top_10_ratings
})

# Merge with the movies dataframe to get the movie titles
recommended_movies = pd.merge(recommended_movies, movies, on='movieId')

print("Top 10 movie recommendations for userId 1:")
print(recommended_movies[['movieId', 'title', 'predicted_rating']])